In [4]:
import re
import mmcv
from sklearn.metrics import average_precision_score
import numpy as np
from typing import Tuple

def get_epoch(file_path: str):
# Read the file
  with open(file_path, "r") as file:
      content = file.read()

  # Use regex to extract the number after "mAP:"
  match = re.search(r"mAP:\s*(\d+)", content)
  if match:
      epoch = int(match.group(1))
      return epoch
  else:
      print("Epoch not found.")

def get_gt_pred(file_path: str):
    output_and_gt = mmcv.load(file_path)
    return output_and_gt[0]['outputs'], output_and_gt[0]['gt_labels']
import numpy as np
outputs_result = None
gt_labels_result = None
for i in range(1,5):
    file_path = f"coco/coco_uniform_bce_group{i}.txt"
    epoch = get_epoch(file_path)
    outputs, gt_labels = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_uniform_bce_group{i}/gt_and_results_e{epoch}.pkl")
    if outputs_result is None:
        outputs_result = outputs
    else:
        outputs_result = np.hstack((np.array(outputs_result), np.array(outputs)))
    if gt_labels_result is None:
        gt_labels_result = gt_labels
    else:
        gt_labels_result = np.hstack((np.array(gt_labels_result), np.array(gt_labels)))
    

def calculate_mAP(all_preds: np.ndarray, all_labels: np.ndarray) -> Tuple[np.float64, list]:
    label_cnt = len(all_labels[0])
    mAP = 0
    mAP_per_label = []
    print(label_cnt)
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    for i in range(label_cnt):
        # print(np.sum(all_labels[:, i]) > 0)
        AP = average_precision_score(all_labels[:, i], all_preds[:, i])
        mAP_per_label.append(AP)
        mAP += AP

    mAP /= label_cnt
    APs = average_precision_score(all_labels, all_preds, average=None) 
    mAP = APs.mean()

    return mAP, mAP_per_label
def get_3_split_map(outputs_result, gt_labels_result):
    map_score1 =  calculate_mAP( np.array(outputs_result)[:, :22], np.array(gt_labels_result)[:, :22])
    # print(f"head mAP: {map_score1[0]:.4f}")
    # print(np.array(map_score[1]))
    map_score2 =  calculate_mAP( np.array(outputs_result)[:, 22:55], np.array(gt_labels_result)[:, 22:55])
    # print(f"mid mAP: {map_score2[0]:.4f}")
    # print(np.array(map_score[1]))
    map_score3 =  calculate_mAP( np.array(outputs_result)[:, 55:], np.array(gt_labels_result)[:, 55:])

    total = calculate_mAP( np.array(outputs_result), np.array(gt_labels_result))
    # print(f"tail mAP: {map_score3[0]:.4f}")
    print([round(map_score1[0], 4), round(map_score2[0], 4), round(map_score3[0], 4), round(total[0], 4)])
    # print(np.array(map_score[1]))


get_3_split_map(outputs_result, gt_labels_result)

/home/mark/Desktop/工研院/DistributionBalancedLoss/.venv/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


22
33
25
80
[0.5759, 0.6518, 0.6167, 0.62]


In [1]:
map_score = {
  "all": [0.5704, 0.636, 0.5447, 0.5894],
  "Classaware+bce":[0.5536, 0.6301, 0.5909, 0.5968],
  "Classaware+DBLoss": [0.5323, 0.6099, 0.5637, 0.5741],
  "IR1.5 CVIR0.2":[0.564, 0.6485, 0.6075, 0.6124],
  "IR2.5 CVIR0.4":[0.5759, 0.6518, 0.6167, 0.62],
  "IR3.5, CVIR0.6": [0.5763, 0.6504, 0.6218, 0.6211],
  "split": [0.5755, 0.6527, 0.6177, 0.6205]
}



head mAP: 0.5649
mid mAP: 0.6351
tail mAP: 0.5310
total mAP: 0.5833

head mAP: 0.5668
mid mAP: 0.6352
tail mAP: 0.5581
total mAP: 0.5927

SyntaxError: invalid syntax (1892456545.py, line 11)

In [62]:
# 1_2_3
# Best head mAP: 0.6294 at index 33
# Best mid mAP: 0.5999 at index 77
# Best tail mAP: 0.5674 at index 70
# Best total mAP: 0.5894 at index 77

#1
# Best epoch for total mAP: 8 with score: 0.6302

# 1_2
# Best head mAP: 0.6231 at index 18
# Best mid mAP: 0.5985 at index 37

# 2
# Best epoch for total mAP: 30 with score: 0.6084

# 2_3
# Best mid mAP: 0.6058 at index 52
# Best tail mAP: 0.5921 at index 68

# 3
# Best epoch for total mAP: 58 with score: 0.6344

In [8]:
(0.5755*22+ 0.6527*33+ 0.6177*25) /80

0.6205325

In [2]:
(0.5763*22+0.6504*33+ 0.6218*25)/80 #0.6211

0.6210850000000001

In [58]:
import pandas as pd
original = list(pd.read_csv("/home/mark/Desktop/工研院/multi-label_classification/data/coco/train_data.csv").columns[1:])
split = []
for i in range(1, 4):
    file_path = f"/home/mark/Desktop/工研院/multi-label_classification/data/coco/group{i}_train_data.csv"
    group = pd.read_csv(file_path).columns[1:]
    split.append(group)


    
indexed_lists = [
    [original.index(item) for item in sublist]
    for sublist in split
]

print(indexed_lists)

[[0, 60, 56, 41, 39, 45, 73, 2, 71, 43, 62], [26, 69, 44, 58, 57, 63, 67, 42, 72, 75, 24, 13, 68, 65, 7, 74, 40, 55, 66, 16, 64, 9, 15, 53, 48, 59, 5, 25, 46, 1, 49, 32, 47, 3, 27, 50, 34, 51, 61], [28, 35, 77, 8, 38, 14, 6, 17, 36, 54, 76, 52, 11, 30, 10, 29, 79, 19, 33, 37, 4, 70, 12, 20, 31, 18, 22, 23, 21, 78]]


In [57]:
original

Index(['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
       'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
       'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
       'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
       'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
       'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
       'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
       'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
       'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
       'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
       'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
       'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
       'scissors', 'teddy bear', 'hair drier', 'toothbrush'],
      dtype='object')

In [13]:
epoch_list = []

for i in range(1,9):
    # epoch = get_epoch(file_path)
    outputs, gt_labels = get_gt_pred(f"work_dirs/LT_coco_resnet50_pfc_DB_classaware_DBloss/gt_and_results_e{i}.pkl")
    epoch_list.append((outputs, gt_labels))


    

In [ ]:
for outputs, gt_labels in epoch_list:
    map_score1 =  calculate_mAP( np.array(outputs)[:, :11], np.array(gt_labels)[:, :11])
    # print(f"head mAP: {map_score1[0]:.4f}")
    # print(np.array(map_score[1]))
    map_score2 =  calculate_mAP( np.array(outputs)[:, 11:50], np.array(gt_labels)[:, 11:50])
    # print(f"mid mAP: {map_score2[0]:.4f}")
    # print(np.array(map_score[1]))
    map_score3 =  calculate_mAP( np.array(outputs)[:, 50:], np.array(gt_labels)[:, 50:])

In [14]:
import numpy as np
import mmcv

best_head = (-1, -1)  # (score, index)
best_mid = (-1, -1)
best_tail = (-1, -1)
best_total = (-1, -1)
class_split = mmcv.load("appendix/coco/longtail2017/class_split.pkl")

for idx, (outputs, gt_labels) in enumerate(epoch_list):
    outputs = np.array(outputs)
    gt_labels = np.array(gt_labels)
    print(outputs.shape)

    head_score = calculate_mAP(outputs[:, list(class_split["head"])], gt_labels[:, list(class_split["head"])])[0]
    mid_score = calculate_mAP(outputs[:, list(class_split["middle"])], gt_labels[:, list(class_split["middle"])])[0]
    tail_score = calculate_mAP(outputs[:, list(class_split["tail"])], gt_labels[:, list(class_split["tail"])])[0]
    total_score = calculate_mAP(outputs, gt_labels)[0]

    # head_score = calculate_mAP(outputs[:, :11], gt_labels[:, :11])[0]
    # mid_score = calculate_mAP(outputs[:, 11:50], gt_labels[:, 11:50])[0]
    # tail_score = calculate_mAP(outputs[:, 50:], gt_labels[:, 50:])[0]
    # total_score = calculate_mAP(outputs, gt_labels)[0]

    # head_score = calculate_mAP(outputs[:, indexed_lists[0]], gt_labels[:, indexed_lists[0]])[0]
    # mid_score = calculate_mAP(outputs[:, indexed_lists[1]], gt_labels[:, indexed_lists[1]])[0]
    # tail_score = calculate_mAP(outputs[:, indexed_lists[2]], gt_labels[:, indexed_lists[2]])[0]
    # total_score = calculate_mAP(outputs, gt_labels)[0]

    if head_score > best_head[0]:
        best_head = (head_score, idx)
    if mid_score > best_mid[0]:
        best_mid = (mid_score, idx)
    if tail_score > best_tail[0]:
        best_tail = (tail_score, idx)
    if total_score > best_total[0]:
        best_total = (total_score, idx)

print(f"Best head mAP: {best_head[0]:.4f} at index {best_head[1]}")
print(f"Best mid mAP: {best_mid[0]:.4f} at index {best_mid[1]}")
print(f"Best tail mAP: {best_tail[0]:.4f} at index {best_tail[1]}")
print(f"Best total mAP: {best_total[0]:.4f} at index {best_total[1]}")

(5000, 80)
22
33
25
80
(5000, 80)
22
33
25
80
(5000, 80)
22
33
25
80
(5000, 80)
22
33
25
80
(5000, 80)
22
33
25
80
(5000, 80)
22
33
25
80
(5000, 80)
22
33
25
80
(5000, 80)
22
33
25
80
Best head mAP: 0.5341 at index 6
Best mid mAP: 0.6099 at index 5
Best tail mAP: 0.5637 at index 5
Best total mAP: 0.5741 at index 5


In [37]:
print(f"Best head mAP: {best_head[0]:.4f} at index {best_head[1]}")
print(f"Best mid mAP: {best_mid[0]:.4f} at index {best_mid[1]}")
print(f"Best tail mAP: {best_tail[0]:.4f} at index {best_tail[1]}")
print(f"Best total mAP: {best_total[0]:.4f} at index {best_total[1]}")

Best head mAP: 0.5780 at index 43
Best mid mAP: 0.6364 at index 73
Best tail mAP: 0.5460 at index 70
Best total mAP: 0.5894 at index 77


In [35]:
class_split["middle"]

{1,
 3,
 5,
 7,
 8,
 9,
 13,
 15,
 16,
 25,
 27,
 28,
 32,
 34,
 35,
 38,
 40,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 55,
 59,
 61,
 64,
 65,
 66,
 68,
 74,
 77}

In [15]:
outputs, gt_labels =epoch_list[5]
outputs = np.array(outputs)
gt_labels = np.array(gt_labels)
map_score1 =  calculate_mAP(outputs[:, list(class_split["head"])], gt_labels[:, list(class_split["head"])])
print(f"head mAP: {map_score1[0]:.4f}")
# print(np.array(map_score[1]))
map_score2 =  calculate_mAP(outputs[:, list(class_split["middle"])], gt_labels[:, list(class_split["middle"])])
print(f"mid mAP: {map_score2[0]:.4f}")
# print(np.array(map_score[1]))
map_score3 =  calculate_mAP(outputs[:, list(class_split["tail"])], gt_labels[:, list(class_split["tail"])])
print(f"mid mAP: {map_score3[0]:.4f}")
total = calculate_mAP( np.array(outputs), np.array(gt_labels))
    # print(f"tail mAP: {map_score3[0]:.4f}")
print([round(map_score1[0], 4), round(map_score2[0], 4), round(map_score3[0], 4), round(total[0], 4)])

22
head mAP: 0.5323
33
mid mAP: 0.6099
25
mid mAP: 0.5637
80
[0.5323, 0.6099, 0.5637, 0.5741]


In [27]:
import mmcv
mmcv.load("appendix/coco/longtail2017/class_split.pkl")

{'head': {0,
  2,
  24,
  26,
  39,
  41,
  42,
  43,
  44,
  45,
  56,
  57,
  58,
  60,
  62,
  63,
  67,
  69,
  71,
  72,
  73,
  75},
 'middle': {1,
  3,
  5,
  7,
  8,
  9,
  13,
  15,
  16,
  25,
  27,
  28,
  32,
  34,
  35,
  38,
  40,
  46,
  47,
  48,
  49,
  50,
  51,
  53,
  55,
  59,
  61,
  64,
  65,
  66,
  68,
  74,
  77},
 'tail': {4,
  6,
  10,
  11,
  12,
  14,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  29,
  30,
  31,
  33,
  36,
  37,
  52,
  54,
  70,
  76,
  78,
  79}}

In [67]:
online_data_root = '/home/mark/Desktop/工研院/multi-label_classification/data/coco/DB/'
import mmcv 
annotations = mmcv.load(online_data_root + 'coco_annotations.pkl')

In [68]:
gt = []
for a in annotations:
  gt.append(a['ann']['labels'])

In [74]:
class_freq = mmcv.load("appendix/coco/longtail2017/class_freq.pkl")

In [76]:
class_freq_gt = class_freq['gt_labels']

In [77]:
class_freq_gt[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [85]:
class_freq

{'gt_labels': [array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
  array([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
   

In [86]:

def calculate_ratio(array: np.array, base_index: int, target_index: int):
    # Ensure the array is a 2D numpy array
    array = np.array(array)

    # Count of indices where both array[0][i] == 1 and array[1][i] == 1
    shared_count = np.sum((array[:,base_index] == 1) & (array[:,target_index] == 1))

    # Count of indices where array[0][i] == 1
    total_count = np.sum(array[:,base_index] == 1)

    # Avoid division by zero
    if total_count == 0:
        return 0
    else:
        return shared_count / total_count
def calculate_ratio_base_index(array: np.array, base_index: int):
    array = np.array(array)
    return np.array([calculate_ratio(array, base_index, i) for i in range(array.shape[1])])

# used for create "condition_prob" of class_freq.pkl"
# input is the gt_labels
def calculate_ratio_all(array: np.array):
    array = np.array(array)
    return np.array([calculate_ratio_base_index(array, i) for i in range(array.shape[1])])


gt = np.array(gt)

data = {}


data["gt_labels"] = gt.tolist()

data["class_freq"] = np.sum(gt == 1, axis=0)

data["neg_class_freq"] = np.sum(gt== 0, axis=0)

data["condition_prob"] = calculate_ratio_all(gt)


In [87]:
data

{'gt_labels': [[1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
 

In [ ]:
'condition_prob': array([[1.        , 0.02659574, 0.17464539, ..., 0.00797872, 0.00177305,
         0.0035461 ],
        [0.81081081, 1.        , 0.54054054, ..., 0.        , 0.        ,
         0.        ],
        [0.71376812, 0.07246377, 1.        , ..., 0.00362319, 0.        ,
         0.        ],
        ...,
        [0.42857143, 0.        , 0.04761905, ..., 1.        , 0.        ,
         0.        ],
        [0.33333333, 0.        , 0.        , ..., 0.        , 1.        ,
         0.33333333],
        [0.36363636, 0.        , 0.        , ..., 0.        , 0.18181818,
         1.        ]])}